In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
num_epochs = 10
k_folds = 5

transform_plain = transforms.Compose([
    transforms.ToTensor(),
])

transform_augmented = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.ToTensor(),
])

train_dataset_aug   = datasets.CIFAR10(root='./cifar10', train=True, transform=transform_augmented, download=True)
test_dataset        = datasets.CIFAR10(root='./cifar10', train=False, transform=transform_plain)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.dropout = nn.Dropout(p=dropout_rate)  # Dropout layer

        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, 8 * 8 * 16)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)  # 🔸 Dropout after fc1

        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)  # 🔸 Dropout after fc2

        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)

        return x


In [14]:
# Training function
def train(model, loader, optimizer, criterion, epoch):
    model.train()   # Set the model to training mode

    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(loader):
        # Move data and target to the device
        data, target = data.to(device), target.to(device)   
        optimizer.zero_grad()               # Zero the gradients
        output = model(data)                # Forward pass
        loss = criterion(output, target)    # Compute loss
        loss.backward()                     # Backward pass
        optimizer.step()                    # Update weights

        train_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)

        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, 
                batch_idx * len(data), 
                len(loader.dataset),
                100. * batch_idx / len(loader), 
                loss.item()))
        
    avg_loss = train_loss / total
    accuracy = correct / total
    print(f'Train set: Average loss: {avg_loss:.4f}, Accuracy: {correct}/{total} ({100. * accuracy:.2f}%)')

In [15]:
# Validation function 
def validate(model, loader, criterion):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += data.size(0)
            
    val_loss /= total
    val_acc = correct / total
    print(f'Validation set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{total} '
          f'({100. * val_acc:.0f}%)')

In [16]:
if __name__ == '__main__':
    # K-Fold Cross-Validation
    # Create K-Fold cross-validator with shuffling 
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42) 

    for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset_aug)):
        print(f'\n======================')
        print(f'Fold {fold + 1}/{k_folds}')
        print(f'======================')

        # Subset samplers
        train_subset = Subset(train_dataset_aug, train_idx)
        val_subset = Subset(train_dataset_aug, val_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        # Model, Optimizer, Loss
        model = CNNNet().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),
                                        lr=1e-4, 
                                        betas=(0.9, 0.999),
                                        eps=1e-08,
                                        weight_decay=0,
                                        amsgrad=False)
        

        for epoch in range(num_epochs):
            print(f'\n--- Epoch {epoch + 1} ---')
            train(model, train_loader, optimizer, criterion, epoch)
            validate(model, val_loader, criterion)

    # Optional final test
    print("\n\n Final Evaluation on Test Set")
    model.eval()
    test_correct = 0
    test_total = 0
    test_loss = 0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1)
            test_correct += pred.eq(target).sum().item()
            test_total += data.size(0)

    test_loss /= test_total
    test_acc = test_correct / test_total
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {test_correct}/{test_total} ({100. * test_acc:.2f}%)')


Fold 1/5

--- Epoch 1 ---
Train Epoch: 0 | Batch Status: 0/40000 (0%) | Loss: 2.377940
Train Epoch: 0 | Batch Status: 640/40000 (2%) | Loss: 2.372963
Train Epoch: 0 | Batch Status: 1280/40000 (3%) | Loss: 2.325651
Train Epoch: 0 | Batch Status: 1920/40000 (5%) | Loss: 2.334165
Train Epoch: 0 | Batch Status: 2560/40000 (6%) | Loss: 2.314474
Train Epoch: 0 | Batch Status: 3200/40000 (8%) | Loss: 2.307177
Train Epoch: 0 | Batch Status: 3840/40000 (10%) | Loss: 2.320831
Train Epoch: 0 | Batch Status: 4480/40000 (11%) | Loss: 2.244745
Train Epoch: 0 | Batch Status: 5120/40000 (13%) | Loss: 2.272562
Train Epoch: 0 | Batch Status: 5760/40000 (14%) | Loss: 2.215390
Train Epoch: 0 | Batch Status: 6400/40000 (16%) | Loss: 2.202158
Train Epoch: 0 | Batch Status: 7040/40000 (18%) | Loss: 2.250587
Train Epoch: 0 | Batch Status: 7680/40000 (19%) | Loss: 2.185302
Train Epoch: 0 | Batch Status: 8320/40000 (21%) | Loss: 2.073926
Train Epoch: 0 | Batch Status: 8960/40000 (22%) | Loss: 2.093336
Train Ep